In [1]:
import neptune.new as neptune
import os
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import CyclicLR
import torch.multiprocessing as mp
import numpy as np
import random

import sys
sys.path.append("..") # adds higher directory to python modules path

from LoaderPACK.Unet import Unet
from LoaderPACK.Loader import load_whole_data, load_5_min_intervals, load_shuffle_5_min
from LoaderPACK.Accuarcy_finder import Accuarcy_find_tester
from multiprocessing import Process
from LoaderPACK.trainer import net_train
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import csv

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



# Set up the datasets
random.seed(42)



#val_set, train_set = torch.utils.data.random_split(
#                            random.sample(range(1, 226 + 1), 200), [26, 200],
#                            generator=torch.Generator().manual_seed(42))

val_set, train_set = torch.utils.data.random_split(
                            random.sample(range(1, 226 + 1), 50), [10, 40],
                            generator=torch.Generator().manual_seed(42))


set = [i for i in val_set]

load_file = load_whole_data(path = "C:/Users/Marc/Desktop/model_data",
                                  ind = val_set)

file_loader = torch.utils.data.DataLoader(load_file,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=0)

batch_size = 1

i = 1

valid_acc = torch.tensor([]).to(device)

model = Unet(n_channels = 1, n_classes = 2)
x = torch.load(f"C:/Users/Marc/Desktop/network/SGD_net{i}.pt")
model.load_state_dict(x)
model.to(device)



cuda


Unet(
  (inc): Double_Convolution(
    (conv1): Conv1d(1, 10, kernel_size=(3,), stride=(1,))
    (conv2): Conv1d(10, 10, kernel_size=(3,), stride=(1,))
    (norm1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (norm2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (down1): Down_Scale(
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (doub): Double_Convolution(
      (conv1): Conv1d(10, 20, kernel_size=(3,), stride=(1,))
      (conv2): Conv1d(20, 20, kernel_size=(3,), stride=(1,))
      (norm1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (norm2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
  )
  (down2): Down_Scale(
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (doub): Double_Convolution(
      (conv1

In [4]:
j = 0

tot_sr_nr = 1

for file in file_loader:

    #figure, axis = plt.subplots(8, 1)
#
    ## The artifacts on channel: FP1-F7
    #axis[0].plot(file[0][0][0])
    #axis[0].axes.xaxis.set_visible(False) # remove digits on x-axis
#
    ## The artifacts on channel: F7-T3
    #axis[1].plot(file[1][0][0], "b")
    #axis[1].axes.xaxis.set_visible(False) # remove digits on x-axis
#
    #axis[2].plot(file[0][0][1])
    #axis[2].axes.xaxis.set_visible(False) # remove digits on x-axis
#
    ## The artifacts on channel: F7-T3
    #axis[3].plot(file[1][0][1], "b")
    #axis[3].axes.xaxis.set_visible(False) # remove digits on x-axis
#
#
    #axis[4].plot(file[0][0][2])
    #axis[4].axes.xaxis.set_visible(False) # remove digits on x-axis
#
    ## The artifacts on channel: F7-T3
    #axis[5].plot(file[1][0][2], "b")
    #axis[5].axes.xaxis.set_visible(False) # remove digits on x-axis
#
    #axis[6].plot(file[0][0][3])
    #axis[6].axes.xaxis.set_visible(False) # remove digits on x-axis
#
    ## The artifacts on channel: F7-T3
    #axis[7].plot(file[1][0][3], "b")
    #axis[7].axes.xaxis.set_visible(False) # remove digits on x-axis
#
    #plt.show()


    load_series = load_5_min_intervals(file, device)

    series_loader = torch.utils.data.DataLoader(load_series,
                                                batch_size=batch_size,
                                                shuffle=True,
                                                num_workers=0)
    v_mat = torch.zeros(2,2)
    total_pos, total_neg = torch.tensor(0), torch.tensor(0)

    for series in series_loader:
        ind, tar, chan, cut = series
        y_pred = model(ind)
        #pred = y_pred.transpose(1, 2).reshape(-1, 2).type(fl)
        #target = tar.view(-1).type(it)

        acc, mat, tot_p_g, tot_n_g, art_pred = Accuarcy_find_tester(y_pred, tar, device)
        #valid_acc = torch.cat((valid_acc, acc.view(1)))
        #v_mat = v_mat + mat
        #total_pos = total_pos + tot_p_g
        #total_neg = total_neg + tot_n_g

        #with open("C:/Users/Marc/Desktop/info/data_encoding.csv", "w", newline='') as f:
        #    write = csv.writer(f) # save information that link the nr of the
        #                          # .pt files with the .edf files.
        #    write.writerow(1*art_pred[0].cpu())

        if True:
            figure, axis = plt.subplots(2, 1)

            # The artifacts on channel: FP1-F7
            axis[0].plot(ind[0][0].cpu())
            axis[0].axes.xaxis.set_visible(False) # remove digits on x-axis

            # The artifacts on channel: F7-T3
            axis[1].plot(1*art_pred[0].cpu(), "b")
            axis[1].plot(tar[0][0].cpu(), "r", markersize=1)
            axis[1].set_title(f"Nr:{set[j]} with channel: {chan} at {cut}")
            axis[1].axes.xaxis.set_visible(False) # remove digits on x-axis

            plt.savefig(f"C:/Users/Marc/Desktop/model_plots/graf ({tot_sr_nr}).jpg")
            plt.close()
            tot_sr_nr += 1

    j += 1

# plt.plot(valid_acc.cpu())
# plt.show()


KeyboardInterrupt: 

In [6]:
import sys
sys.path.append("..")
import torch
from LoaderPACK.tester import val_tester
from LoaderPACK.Unet_leaky import Unet_leaky_lstm_elec
import torch.nn as nn
import torch

In [7]:
batch_size = 1
device = "cpu"

model = Unet_leaky_lstm_elec(n_channels=1, batch_size=batch_size, device=device).to(device)

x = torch.load(r"C:\Users\Marc\Desktop\wierd\network_SGD-epk-5.pt", map_location='cpu')

model.load_state_dict(x)
model.to(device)

Unet_leaky_lstm_elec(
  (inc): Double_Convolution(
    (conv1): Conv1d(1, 20, kernel_size=(3,), stride=(1,))
    (conv2): Conv1d(20, 20, kernel_size=(3,), stride=(1,))
    (norm1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (norm2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): LeakyReLU(negative_slope=0.18181818181818182)
  )
  (down1): Down_Scale(
    (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (doub): Double_Convolution(
      (conv1): Conv1d(20, 40, kernel_size=(3,), stride=(1,))
      (conv2): Conv1d(40, 40, kernel_size=(3,), stride=(1,))
      (norm1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (norm2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.18181818181818182)
    )
  )
  (down2): Down_Scale(
    (pool): MaxPool1d(kernel_size=2, 

In [8]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[[  1.0788,  -0.2454,  -0.7581]],

        [[  3.6203,  -4.7367,  -4.5449]],

        [[ -0.1516,   0.7299,   0.3077]],

        [[ -1.9096,  -0.2614,  -0.8491]],

        [[  0.1189,   1.5198,  -1.4154]],

        [[  1.0983,  -0.4599,  -1.6896]],

        [[  0.2916,  -1.3161,   0.2576]],

        [[ 13.3470,  -2.2533,  15.5714]],

        [[ -2.1096,  -1.4146,  -1.9634]],

        [[  0.7188, -25.8170, -13.1760]],

        [[ -0.2015, -28.5131,  -0.4914]],

        [[  1.9504,   0.1525,  -4.8761]],

        [[ -1.0523,   0.2630,   2.1246]],

        [[-20.7841,  23.5894, -21.4894]],

        [[ -0.4839,   0.5842,  -1.5273]],

        [[-20.8620,  -6.3397,   7.8834]],

        [[ -4.9802,   5.8208,   5.7127]],

        [[ -5.6702,   4.8400,   0.4626]],

        [[ -0.4306,   0.8700,  -0.0460]],

        [[ 22.4607,  -3.2254,   7.6034]]], requires_grad=True)
Parameter containing:
tensor([ 0.1782,  0.5226, -0.0511, -0.2317,  0.0589,  0.4928,  0.2654,  0.21